In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohammedabdeldayem/the-fake-or-real-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/the-fake-or-real-dataset


In [3]:
import os
import librosa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed, BatchNormalization
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.layers import Reshape

# Define dataset path (automatically mounted by Kaggle)
DATASET_PATH = "/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training"


In [4]:
def extract_features(file_path, sr=22050, n_mfcc=40):
    y, sr = librosa.load(file_path, sr=sr)

    # Compute Mel-Spectrogram
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Compute MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    
    # Normalize and combine features
    features = np.vstack([mel_spec[:40], mfcc])
    
    return features.T  # Transpose for shape (Time, Features)

# Load dataset (Real and Fake samples)
def load_dataset(directory):
    X, y = [], []
    for label in ["real", "fake"]:
        path = os.path.join(directory, label)
        for file in os.listdir(path):
            if file.endswith(".wav"):
                features = extract_features(os.path.join(path, file))
                X.append(features)
                y.append(label)
    
    return np.array(X), np.array(y)

# Load the dataset
X, y = load_dataset(DATASET_PATH)


In [5]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X = np.array([scaler.fit_transform(x) for x in X])

# Reshape for CNN-LSTM (samples, time steps, features, 1)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert labels to categorical format
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed, Reshape, BatchNormalization, Input

# Define Input Layer
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3], 1))  

# CNN Feature Extractor
x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(input_layer)
x = TimeDistributed(BatchNormalization())(x)



x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)
x = TimeDistributed(BatchNormalization())(x)



x = TimeDistributed(Flatten())(x)

# Reshape before LSTM
x = Reshape((X_train.shape[1], -1))(x)  #  Ensures correct shape

# LSTM for Sequence Modeling
x = LSTM(64, return_sequences=False)(x)

# Fully Connected Layers
x = Dense(64, activation="relu")(x)
x = Dropout(0.3)(x)
output_layer = Dense(2, activation="softmax")(x)  # Output: Real or Fake

# Define Model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile Model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train Model
history = model.fit(X_train, y_train, epochs=30)


Epoch 1/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 129s 243ms/step - accuracy: 0.6976 - loss: 0.5611
Epoch 2/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 244ms/step - accuracy: 0.8859 - loss: 0.2734
Epoch 3/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 244ms/step - accuracy: 0.8978 - loss: 0.2462
Epoch 4/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 243ms/step - accuracy: 0.9454 - loss: 0.1450
Epoch 5/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 243ms/step - accuracy: 0.9458 - loss: 0.1398
Epoch 6/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 244ms/step - accuracy: 0.9525 - loss: 0.1305
Epoch 7/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 244ms/step - accuracy: 0.9613 - loss: 0.1036
Epoch 8/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 244ms/step - accuracy: 0.9731 - loss: 0.0696
Epoch 9/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 244ms/step - accuracy: 0.9725 - loss: 0.0800
Epoch 10/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 243ms/step - accuracy: 0.9770 - loss: 0.0631
Epoch 11/30
349/349 ━━━━━━━━━━━━━━━━━━━━ 85s 243ms/step - accuracy: 0.9806 - loss: 0.0542
Epoch 12/30
349/34

In [9]:
acc=model.evaluate(X_test,y_test,verbose=0)[1]
print(acc)

0.9720630645751953
